**ML PROBLEM 5.1**

In [2]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LassoCV, Lasso
from sklearn import linear_model 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *
import warnings
import os
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score
warnings.filterwarnings("ignore")

In [4]:
X_train = np.load(os.path.abspath('Xtrain_Classification1.npy'))
y_train = np.load(os.path.abspath('ytrain_Classification1.npy'))
X_test  = np.load(os.path.abspath('Xtest_Classification1.npy'))

In [8]:
print(X_train.shape)
# 6254 elements with 2352 attributes
print(X_test.shape)
print(y_train.shape)


#Checking the imbalance in the data
y_melanoma=[y for y in y_train if y]
y_nevu=[y for y in y_train if not y]

print(len(y_melanoma))
print(len(y_nevu))

(6254, 2352)
(1764, 2352)
(6254,)
896
5358


prof da cadeira diz pa usar transfer learning


podemos só usar classificadores do sklearn tipo mlpclassifier, logistic regression, naive bayes, svm

balanced accuracy: (sensitivity + setificity)/2
sensitivity= True positive/(true positvity + false negativity)
setificity = (true negative)/(true neg + false positive)


Use pytorch neural network - harder but most usefull for harder problems (other options are jax and keras)
output layer - 2 neurons is a classification problem - use softmax (if it was 1 neuron it would be regression)



In [ ]:
#balanced accuracy


In [6]:
# keras based model
import tensorflow as tf
import keras
from keras import layers


model_keras = keras.Sequential()
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

model_keras.add(layer1)
model_keras.add(layer2)
model_keras.add(layer3)
model_keras.add(layers.Dense(2, activation="softmax"))


